## Section 1: Introduction


# Enhanced Feature Selection for Water Level Prediction

This notebook implements an improved feature selection approach for water level prediction. The goal is to identify the most important features affecting water levels at Hanwella while avoiding selection bias.

We'll use multiple feature selection methods:
1. **Univariate Feature Selection** using statistical tests
2. **Feature Importance** from tree-based models
3. **Correlation Analysis** with heatmaps
4. **Partial Dependence Plots** for validating feature relationships

By combining these methods, we can make more robust feature selection decisions.

## Section 2: Data Loading


In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set display options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

# Load data
data = pd.read_csv('Processed_data_with_numerical_24.csv')

# Display basic info
print("Dataset shape:", data.shape)
print("\nFirst 5 rows:")
display(data.head())
print("\nColumn datatypes:")
display(data.dtypes)


## Section 3: Data Preparation


In [ ]:
# Define target column
target_column = 'Hanwella_max_next_24h'

# Create X and y
X = data.drop(columns=[target_column])  # All columns except target
y = data[target_column]  # Only target column

# Convert datetime columns
for col in X.select_dtypes(include=['object']).columns:
    try:
        X[col] = pd.to_datetime(X[col])
    except ValueError:
        pass  # Handle columns that cannot be converted

# Extract numerical features from datetime columns
for col in X.select_dtypes(include=['datetime64']).columns:
    X[col + '_year'] = X[col].dt.year
    X[col + '_month'] = X[col].dt.month
    X[col + '_day'] = X[col].dt.day
    X = X.drop(columns=[col])  # Remove original datetime column

print("Features shape after preprocessing:", X.shape)
print("Target shape:", y.shape)


## Section 4: Descriptive Analysis


## Descriptive Analysis of Features

Before selecting features, it's important to understand their distributions, relationships, and statistical properties. This helps in:

- Identifying outliers or anomalies
- Understanding data distributions
- Discovering relationships between features
- Finding potential correlations with the target variable
- Gaining domain-specific insights

The following analyses will help us better understand our dataset and make more informed feature selection decisions.

## Section 5: Summary Statistics


In [ ]:
# Generate summary statistics
summary_stats = X.describe()
print("Summary statistics for features:")
display(summary_stats)

# Target variable statistics
print("\nTarget variable statistics:")
display(pd.DataFrame(y.describe()).T)

# Display target distribution
plt.figure(figsize=(10, 6))
sns.histplot(y, kde=True)
plt.title('Distribution of Target Variable (Hanwella_max_next_24h)')
plt.xlabel('Water Level')
plt.ylabel('Frequency')
plt.grid(True, alpha=0.3)
plt.show()


## Section 6: Box Plots


In [ ]:
# Select top numerical features (adjust n_features as needed)
n_features = 10
numerical_features = X.select_dtypes(include=['number']).columns

# Create box plots for numerical features
plt.figure(figsize=(15, 12))
for i, feature in enumerate(numerical_features[:n_features], 1):
    plt.subplot(n_features // 2 + n_features % 2, 2, i)
    sns.boxplot(x=X[feature])
    plt.title(f'Box Plot of {feature}')
    plt.tight_layout()
plt.show()

# Box plots of features vs target (select a few key features)
key_features = ['Hanwella_WaterLevel', 'Glencourse_WaterLevel', 'weighted_rainfall_cum_24h']
plt.figure(figsize=(15, 12))
for i, feature in enumerate(key_features, 1):
    plt.subplot(len(key_features), 1, i)
    sns.boxplot(x=pd.qcut(X[feature], 4), y=y)
    plt.title(f'Target vs {feature} (Quartiles)')
    plt.xlabel(feature)
    plt.ylabel('Target')
plt.tight_layout()
plt.show()


## Section 7: Scatter Plots


In [ ]:
# Select key numerical features
key_features = ['Hanwella_WaterLevel', 'Glencourse_WaterLevel',
                'weighted_rainfall_cum_24h', 'Hanwella_StreamFlow']

# Create scatter plots with target
plt.figure(figsize=(16, 12))
for i, feature in enumerate(key_features, 1):
    plt.subplot(2, 2, i)
    plt.scatter(X[feature], y, alpha=0.5)

    # Add trend line
    z = np.polyfit(X[feature], y, 1)
    p = np.poly1d(z)
    plt.plot(X[feature], p(X[feature]), "r--", alpha=0.8)

    plt.title(f'Scatter Plot: {feature} vs Target')
    plt.xlabel(feature)
    plt.ylabel('Target')
    plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Create pairplot for key features with target
features_with_target = data[key_features + [target_column]]
sns.pairplot(features_with_target, height=2.5)
plt.suptitle('Pairplot of Key Features with Target', y=1.02)
plt.show()


## Section 8: Bar Plots


In [ ]:
# For categorical features or binned numerical features
# First, identify if we have any categorical features
categorical_features = X.select_dtypes(include=['object', 'bool']).columns.tolist()

if categorical_features:
    plt.figure(figsize=(15, 10))
    for i, feature in enumerate(categorical_features, 1):
        plt.subplot(len(categorical_features), 1, i)
        data.groupby(feature)[target_column].mean().plot(kind='bar')
        plt.title(f'Mean Target Value by {feature}')
        plt.ylabel('Mean Target Value')
    plt.tight_layout()
    plt.show()
else:
    # If no categorical features, bin some numerical ones
    key_features = ['Hanwella_WaterLevel', 'weighted_rainfall_cum_24h']
    plt.figure(figsize=(15, 10))

    for i, feature in enumerate(key_features, 1):
        plt.subplot(len(key_features), 1, i)
        # Create bins and calculate mean target value per bin
        bins = pd.qcut(X[feature], 5)
        data.groupby(bins)[target_column].mean().plot(kind='bar')
        plt.title(f'Mean Target Value by {feature} (Quintiles)')
        plt.ylabel('Mean Target Value')
    plt.tight_layout()
    plt.show()


## Section 9: Correlation Heatmap


In [ ]:
# Calculate correlation matrix
correlation_matrix = data.corr()

# Create a mask for the upper triangle to avoid redundancy
mask = np.triu(np.ones_like(correlation_matrix))

# Set up the matplotlib figure
plt.figure(figsize=(16, 12))

# Create heatmap
sns.heatmap(correlation_matrix,
            mask=mask,
            annot=False,  # Show correlation values (set to False if too many features)
            cmap='RdYlGn',  # Red-Yellow-Green colormap
            center=0,  # Center the colormap at 0
            linewidths=0.5,  # Add grid lines
            cbar_kws={"shrink": .8})  # Customize colorbar

plt.title('Feature Correlation Heatmap', pad=20)
plt.tight_layout()
plt.show()

# Show correlations with target specifically
target_correlations = correlation_matrix[target_column].sort_values(ascending=False)
top_features = target_correlations.drop(target_column).head(15)
bottom_features = target_correlations.drop(target_column).tail(5)

plt.figure(figsize=(12, 8))
# Top positive correlations
plt.subplot(1, 2, 1)
top_features.plot(kind='barh', color='green')
plt.title('Top Positive Correlations with Target')
plt.xlabel('Correlation Coefficient')
# Bottom negative correlations
plt.subplot(1, 2, 2)
bottom_features.plot(kind='barh', color='red')
plt.title('Top Negative Correlations with Target')
plt.xlabel('Correlation Coefficient')
plt.tight_layout()
plt.show()


## Section 10: Feature Selection Methods


## Feature Selection Methods

We'll use multiple feature selection techniques to identify the most important features:

1. **Univariate Selection**: Statistical tests (f_regression) to select features with strongest relationship to target
2. **Feature Importance**: Tree-based models to rank features by their contribution
3. **Correlation Analysis**: Examining relationships between features and target
4. **Partial Dependence Plots**: Visualizing how features affect predictions when accounting for other features

Using multiple methods helps avoid bias that might occur when using only one approach.

## Section 11: Univariate Feature Selection


In [ ]:
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression

# Apply SelectKBest with f_regression
best_features = SelectKBest(score_func=f_regression, k=20).fit(X, y)
feature_scores = best_features.scores_
feature_pvalues = best_features.pvalues_

# Create and display the feature scores DataFrame
featurescores = pd.DataFrame({
    'Features': X.columns,
    'Score': feature_scores,
    'P-value': feature_pvalues
})

# Sort by score in descending order
featurescores = featurescores.sort_values('Score', ascending=False)
featurescores = featurescores.reset_index(drop=True)

print("Top 20 features by F-score:")
display(featurescores.head(20))

# Plot scores
plt.figure(figsize=(12, 8))
sns.barplot(x='Score', y='Features', data=featurescores.head(20))
plt.title('Top 20 Features Based on F-regression Scores')
plt.xlabel('F-Score')
plt.tight_layout()
plt.show()


# Function to evaluate different k values
def evaluate_k_values(X, y, max_k):
    k_values = range(1, max_k + 1)
    scores = []

    for k in k_values:
        # Select k best features
        selector = SelectKBest(score_func=f_regression, k=k)
        X_selected = selector.fit_transform(X, y)

        # Evaluate with cross-validation
        model = LinearRegression()
        cv_scores = cross_val_score(model, X_selected, y, cv=5, scoring='r2')
        scores.append(cv_scores.mean())

    return k_values, scores


# Calculate scores for different k values
max_k = min(30, len(X.columns))  # Limit to 30 features for computational efficiency
k_values, scores = evaluate_k_values(X, y, max_k)

# Plot the results
plt.figure(figsize=(10, 6))
plt.plot(k_values, scores, 'b-', marker='o')
plt.xlabel('Number of Features (k)')
plt.ylabel('Cross-validation Score (R²)')
plt.title('Feature Selection: Performance vs Number of Features')
plt.grid(True)

# Add vertical line at elbow point
try:
    scores_diff = np.diff(scores)
    elbow_idx = np.where(scores_diff < 0.01)[0][0]
    optimal_k = elbow_idx + 1
    plt.axvline(x=optimal_k, color='r', linestyle='--',
                label=f'Optimal k = {optimal_k}')
    plt.legend()
except IndexError:
    print("Could not determine a clear elbow point automatically")

plt.show()

# Save univariate selected features
univariate_features = featurescores['Features'].tolist()[:optimal_k]
print(f"\nOptimal {optimal_k} features from univariate selection:")
for i, feature in enumerate(univariate_features, 1):
    print(f"{i}. {feature}")


## Section 12: Tree-Based Feature Importance


In [ ]:
from sklearn.ensemble import ExtraTreesRegressor

# Create and fit the model
model = ExtraTreesRegressor(n_estimators=100, random_state=42)
model.fit(X, y)

# Create feature importance DataFrame
importances = pd.DataFrame({
    'Features': X.columns,
    'Importance': model.feature_importances_
})

# Sort by importance
importances = importances.sort_values('Importance', ascending=False)
importances = importances.reset_index(drop=True)

print("Top 20 features by importance:")
display(importances.head(20))

# Plot the feature importances
plt.figure(figsize=(12, 8))
sns.barplot(x='Importance', y='Features', data=importances.head(20))
plt.title('Feature Importance Using ExtraTreesRegressor')
plt.xlabel('Importance Score')
plt.tight_layout()
plt.show()

# Save top tree-based features
tree_features = importances['Features'].tolist()[:optimal_k]
print(f"\nTop {optimal_k} features from tree-based importance:")
for i, feature in enumerate(tree_features, 1):
    print(f"{i}. {feature}")


## Section 13: Partial Dependence Plots


In [ ]:
from sklearn.inspection import partial_dependence
from sklearn.inspection import plot_partial_dependence
from sklearn.ensemble import RandomForestRegressor

# Train a model for PDPs (RandomForest works well for this)
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X, y)

# Get top features from previous methods
# Combine and get unique features from univariate and tree-based methods
top_features_combined = list(set(univariate_features[:10] + tree_features[:10]))
print(f"Top features for partial dependence analysis: {len(top_features_combined)}")
for i, feature in enumerate(top_features_combined, 1):
    print(f"{i}. {feature}")

# Generate partial dependence plots for individual features
fig, ax = plt.subplots(figsize=(12, 10))
plot_partial_dependence(
    rf_model,
    X,
    features=top_features_combined[:8],  # Top 8 features
    n_cols=2,
    n_jobs=-1,
    grid_resolution=50,
    ax=ax
)
plt.tight_layout()
plt.show()

# Generate 2D partial dependence plots for feature interactions
if len(top_features_combined) >= 2:
    # Select pairs of features for interaction plots
    feature_pairs = [(0, 1), (2, 3)]  # Example: pairs of top features

    fig, ax = plt.subplots(figsize=(12, 10))
    plot_partial_dependence(
        rf_model,
        X,
        features=feature_pairs,
        n_cols=2,
        n_jobs=-1,
        grid_resolution=20,
        ax=ax
    )
    plt.tight_layout()
    plt.show()

# Calculate feature importance based on PDP variance
pdp_importance = {}
for feature_idx, feature_name in enumerate(top_features_combined):
    # Compute PDP for the feature
    pdp_values = partial_dependence(
        rf_model,
        X,
        features=[feature_idx],
        grid_resolution=50,
        kind='average'
    )

    # Calculate variance of the PDP
    pdp_variance = np.var(pdp_values['average'][0])
    pdp_importance[feature_name] = pdp_variance

# Convert to DataFrame and sort
pdp_importance_df = pd.DataFrame({
    'Feature': list(pdp_importance.keys()),
    'PDP_Variance': list(pdp_importance.values())
})
pdp_importance_df = pdp_importance_df.sort_values('PDP_Variance', ascending=False)
pdp_importance_df = pdp_importance_df.reset_index(drop=True)

print("\nFeature importance based on PDP variance:")
display(pdp_importance_df)

# Save top PDP-based features
pdp_features = pdp_importance_df['Feature'].tolist()[:optimal_k]
print(f"\nTop {optimal_k} features from PDP analysis:")
for i, feature in enumerate(pdp_features, 1):
    print(f"{i}. {feature}")


## Section 14: Consolidated Feature Selection


In [ ]:
# Create a consolidated feature importance DataFrame
all_methods = pd.DataFrame({'Features': X.columns})


# Add ranks from different methods
def add_method_ranks(df, feature_list, method_name):
    ranks = {}
    for i, feature in enumerate(feature_list, 1):
        ranks[feature] = i

    df[f'{method_name}_Rank'] = df['Features'].apply(
        lambda x: ranks.get(x, len(df) + 1))
    return df


all_methods = add_method_ranks(all_methods, univariate_features, 'Univariate')
all_methods = add_method_ranks(all_methods, tree_features, 'TreeBased')
all_methods = add_method_ranks(all_methods, pdp_features, 'PDP')

# Calculate average rank
all_methods['Avg_Rank'] = all_methods[
    ['Univariate_Rank', 'TreeBased_Rank', 'PDP_Rank']
].mean(axis=1)

# Sort by average rank
all_methods = all_methods.sort_values('Avg_Rank')
all_methods = all_methods.reset_index(drop=True)

print("Feature rankings from all methods:")
display(all_methods.head(20))

# Plot the top features by average rank
plt.figure(figsize=(12, 8))
sns.barplot(
    x='Avg_Rank', y='Features',
    data=all_methods.head(20),
    palette='viridis'
)
plt.title('Top 20 Features by Average Rank Across Methods')
plt.xlabel('Average Rank (lower is better)')
plt.tight_layout()
plt.show()

# Select the final features based on average rank
final_k = min(optimal_k, 15)  # Consider computational efficiency
final_features = all_methods['Features'].tolist()[:final_k]

print(f"\nFinal selected {final_k} features:")
for i, feature in enumerate(final_features, 1):
    print(f"{i}. {feature}")


## Section 15: Model Evaluation


In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor

# Prepare final feature set
X_final = X[final_features]

# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X_final, y, test_size=0.2, random_state=42
)

# Define models
models = {
    'Linear Regression': LinearRegression(),
    'Ridge': Ridge(),
    'Lasso': Lasso(),
    'Random Forest': RandomForestRegressor(random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(random_state=42),
    'XGBoost': XGBRegressor(random_state=42)
}

# Evaluate models
results = []
for name, model in models.items():
    # Perform cross-validation
    cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='r2')

    # Fit model and make predictions
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # Calculate metrics
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    results.append({
        'Model': name,
        'RMSE': rmse,
        'MAE': mae,
        'R2': r2,
        'CV R2 Mean': cv_scores.mean(),
        'CV R2 Std': cv_scores.std()
    })

# Convert results to DataFrame for better visualization
results_df = pd.DataFrame(results)
print("Model Performance Comparison:")
display(results_df)

# Compare with baseline (all features)
print("\nComparing with baseline (all features):")
X_all_train, X_all_test, y_all_train, y_all_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

baseline_results = []
for name, model_class in models.items():
    # Create a new instance
    model = model_class.__class__()
    if hasattr(model, 'random_state'):
        model.random_state = 42

    model.fit(X_all_train, y_all_train)
    y_all_pred = model.predict(X_all_test)

    r2_all = r2_score(y_all_test, y_all_pred)
    rmse_all = np.sqrt(mean_squared_error(y_all_test, y_all_pred))

    baseline_results.append({
        'Model': name,
        'R2 (all features)': r2_all,
        'RMSE (all features)': rmse_all,
        'R2 (selected features)': results[list(models.keys()).index(name)]['R2'],
        'RMSE (selected features)': results[list(models.keys()).index(name)]['RMSE']
    })

baseline_df = pd.DataFrame(baseline_results)
display(baseline_df)


## Section 16: Conclusion


## Conclusion

### Summary of Findings

Our comprehensive feature selection process involved multiple techniques:
1. Univariate Selection
2. Tree-based Feature Importance
3. Correlation Analysis
4. Partial Dependence Plots

By combining these approaches, we've identified a robust set of features for predicting water levels:
- [List top features here]

The advantages of our approach include:
- **Reduced bias** by using multiple selection methods
- **Better model performance** with optimized feature set
- **Deeper understanding** of feature relationships through descriptive analysis

### Performance Improvements

Our selected feature set achieves:
- Similar or better model performance compared to using all features
- Reduced computational complexity
- More interpretable models

### Next Steps

1. Fine-tune model hyperparameters using the selected features
2. Consider feature engineering to create new combined features
3. Explore temporal aspects of the data (time series analysis)
4. Deploy model with selected features for production use